## Import Library

In [25]:
import numpy as np
import pandas as pd
import seaborn as sns
import time
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import requests

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120

import warnings
warnings.filterwarnings('ignore')

In [26]:
USER_AGENTS = [
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36"
]

HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
VNDIRECT_DATA_HISTORY_URL = 'https://dchart-api.vndirect.com.vn/dchart/history'
URL = VNDIRECT_DATA_HISTORY_URL

In [27]:
def getStockHistoryData(ticker, timestamp_from=0, timestamp_to=0):
    if timestamp_from == 0:
        three_months = date.today() + relativedelta(months=-3)
        timestamp_from = datetime.strptime(three_months.strftime("%m/%d/%Y") + ', 00:00:0', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()
    if timestamp_to == 0:
        timestamp_to = datetime.strptime(date.today().strftime("%m/%d/%Y") + ', 23:59:00', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()

    params = {
        "resolution": "5",
        "symbol": str(ticker),
        "from": int(timestamp_from),
        "to": int(timestamp_to)
    }

    x = requests.get(URL, params=params, headers=HEADERS)
    response = x.json()

    import numpy as np
    import pandas as pd

    timestamp = np.array(response['t']).astype(int)
    close = np.array(response['c']).astype(float)
    open = np.array(response['o']).astype(float)
    high = np.array(response['h']).astype(float)
    low = np.array(response['l']).astype(float)
    volume = np.array(response['v']).astype(int)

    dataset = pd.DataFrame({'Time': timestamp, 'Open': list(open), 'High': list(high), 'Low': list(low),
                            'Close': list(close), 'Volume': list(volume)},
                           columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    return dataset

In [28]:
def prepareData(htd):
    if 'Time' in htd.columns:
        from datetime import datetime

        htd['DateStr'] = htd.apply(
            lambda x: datetime.fromtimestamp(x['Time']).strftime("%Y-%m-%d %H:%M:%S"), axis=1)

    htd['Date'] = pd.to_datetime(htd['DateStr'])
    # htd['Date'] = htd['Date'] + pd.DateOffset(hours=7)
    ticker_data = htd.set_index('Date')
    ticker_data.drop(columns=['Time', 'DateStr'], inplace=True)
    return ticker_data

## Load Price Data

In [29]:
ticker = "VN30F1M"
htd = getStockHistoryData(ticker, 0, 0)
ticker_data = prepareData(htd)
ticker_data = ticker_data.dropna()
ticker_data

,Open,High,Low,Close,Volume
Date,,,,,
2024-04-10 09:00:00,1265.0,1266.0,1264.2,1265.8,5432
2024-04-10 09:05:00,1266.5,1267.6,1266.5,1267.1,4159
2024-04-10 09:10:00,1267.1,1269.0,1267.0,1268.9,6049
2024-04-10 09:15:00,1268.9,1269.7,1268.5,1268.5,4879
2024-04-10 09:20:00,1268.5,1268.8,1268.0,1268.5,3560
...,...,...,...,...,...
2024-07-10 14:15:00,1314.6,1315.9,1312.1,1314.3,9556
2024-07-10 14:20:00,1314.2,1314.4,1310.3,1310.7,10901
2024-07-10 14:25:00,1310.7,1312.9,1310.0,1310.3,8964


In [30]:
ticker_data.columns = ["open", "high", "low", "close", "volume"]

,open,high,low,close,volume
Date,,,,,
2024-07-10 09:00:00,1318.9,1319.9,1318.6,1319.6,4738
2024-07-10 09:05:00,1319.5,1319.7,1319.0,1319.5,1450
2024-07-10 09:10:00,1319.4,1320.8,1319.2,1319.9,3197
2024-07-10 09:15:00,1319.7,1321.3,1319.7,1320.8,2506
2024-07-10 09:20:00,1320.9,1321.0,1319.3,1319.3,1786
2024-07-10 09:25:00,1319.3,1319.3,1316.3,1316.3,6583
2024-07-10 09:30:00,1316.3,1316.4,1315.0,1316.1,6591
2024-07-10 09:35:00,1316.3,1317.2,1315.9,1317.0,3668
2024-07-10 09:40:00,1317.0,1318.3,1316.4,1317.8,4236


In [31]:
#@title Raw Price
def cal_first_close(tick):
  if not tick.empty:
    return tick[0]


def cal_high(tick):
  tick = tick[100*tick.index.hour+tick.index.minute > 925]
  tick = tick[100*tick.index.hour+tick.index.minute < 1355]
  return tick.max()


def cal_low(tick):
  tick = tick[100*tick.index.hour+tick.index.minute > 925]
  tick = tick[100*tick.index.hour+tick.index.minute < 1355]
  return tick.min()


def cal_price(tick):
  tick = tick[100*tick.index.hour+tick.index.minute == 1355]
  if not tick.empty:
    return tick[0]

def cal_close(tick):
  tick = tick[100*tick.index.hour+tick.index.minute == 1425]
  if not tick.empty:
    return tick[0]

price = ticker_data
price['first_close'] = price.close
price['price'] = price.close
price = price.resample("D").agg({
    'first_close': cal_first_close,
    'high':cal_high,
    'low': cal_low,
    'price': cal_price,
    'close': cal_close
    })
raw = price.dropna()

In [32]:
price = raw.copy()
price.tail(10)

,first_close,high,low,price,close
Date,,,,,
2024-06-27,1285.0,1289.0,1280.8,1286.5,1289.8
2024-06-28,1292.0,1291.2,1281.2,1283.9,1277.9
2024-07-01,1277.5,1279.2,1271.7,1279.4,1284.8
2024-07-02,1286.6,1299.0,1284.6,1297.1,1296.6
2024-07-03,1296.5,1306.5,1294.5,1305.1,1305.5
2024-07-04,1306.7,1313.3,1306.1,1308.3,1308.0
2024-07-05,1309.7,1314.5,1307.2,1308.8,1316.2
2024-07-08,1316.1,1316.5,1305.6,1310.7,1312.0
2024-07-09,1315.1,1320.3,1304.5,1318.2,1319.5


In [33]:
price['percent'] = 100 * (price.price - price.close.shift(1)) / price.close.shift(1)
price['returns'] = (price.price - price.first_close) / (price.high - price.low)
price['return'] = 1000 * (price.close - price.price) / price.price
price.tail(10)

,first_close,high,low,price,close,percent,returns,return
Date,,,,,,,,
2024-06-27,1285.0,1289.0,1280.8,1286.5,1289.8,-0.232648,0.182927,2.565099
2024-06-28,1292.0,1291.2,1281.2,1283.9,1277.9,-0.457435,-0.810000,-4.673261
2024-07-01,1277.5,1279.2,1271.7,1279.4,1284.8,0.117380,0.253333,4.220728
2024-07-02,1286.6,1299.0,1284.6,1297.1,1296.6,0.957347,0.729167,-0.385475
2024-07-03,1296.5,1306.5,1294.5,1305.1,1305.5,0.655561,0.716667,0.306490
2024-07-04,1306.7,1313.3,1306.1,1308.3,1308.0,0.214477,0.222222,-0.229305
2024-07-05,1309.7,1314.5,1307.2,1308.8,1316.2,0.061162,-0.123288,5.654034
2024-07-08,1316.1,1316.5,1305.6,1310.7,1312.0,-0.417870,-0.495413,0.991836
2024-07-09,1315.1,1320.3,1304.5,1318.2,1319.5,0.472561,0.196203,0.986193


### Live signal